# AFF - Generating Randomized Treated Landscape Scenarios
## Creates Randomized Treatment Scenario Landscapes - exports as ee.Images into an ee.ImageCollection

In [1]:
import ee
import os
import random
import numpy as np
import math
from datetime import datetime

from src.utils.yml_params import get_export_params
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize()
%load_ext autoreload
%autoreload 2


## USER INPUT - Review/edit the parameters for the treatment randomization procedure

In [2]:
# range of pct of total areas you want to simulate in scenarios
SCENARIO_PCT_RANGE = [0.0,0.6] # make as static list ascending not randomized b/w range

# distinct acreage units to generate
SIZE_CLASSES = [10,40,100,400]  # must be length 4
                               
DISTRO = 'log' # 'log' or 'norm'

DIST = 322 # Determine what DIST code the treated pixels will have, this will affect the fuel layer update results
## ^^ 331 wasn't in the zone6 master lookup table as a DIST code so using 332

## USER INPUT - Review/edit paths to your AOIs

In [3]:
private_p = ee.FeatureCollection("projects/pyregence-ee/assets/aff-treatments/AFF_private")
private_img = ee.Image("projects/pyregence-ee/assets/aff-treatments/AFF_private_img").gt(0)
public_p = ee.FeatureCollection("projects/pyregence-ee/assets/aff-treatments/AFF_public")
public_img = ee.Image("projects/pyregence-ee/assets/aff-treatments/AFF_public_img").gt(0)

# latest pacel AOIs
private_no_grass = ee.FeatureCollection("projects/pyregence-ee/assets/aff-treatments/AFF_private_nograss") 
public_no_grass = ee.FeatureCollection("projects/pyregence-ee/assets/aff-treatments/AFF_public_nograss")
private_no_grass_img = ee.Image("projects/pyregence-ee/assets/aff-treatments/AFF_private_nograss_img") 
public_no_grass_img = ee.Image("projects/pyregence-ee/assets/aff-treatments/AFF_public_nograss_img") 



### Calculate Area of your AOIs and Generate your Export Region
### here we calc area of public parcels, private parcels, and their total combined area

In [4]:
# generate a region for clipping and exporting encompassing all parcels
total_area_img = private_no_grass_img.add(public_no_grass_img).gte(1)

REGION = ee.FeatureCollection("projects/pyregence-ee/assets/aff-treatments/PC442_2km_buffer_bounds").geometry() # using DS's export bounds shp
# REGION = ee.Image(total_area_img).geometry().bounds()

public_img_area = ee.Image.pixelArea().updateMask(ee.Image().paint(public_no_grass,1))
public_px_area = public_img_area.reduceRegion(**{
'reducer': ee.Reducer.sum(),
'geometry': REGION,
'scale': 5,
'crs': CRS,  
'maxPixels': 1e13
})
AREA_PUBLIC = ee.Number(ee.Dictionary(public_px_area).get('area')).getInfo()
print(AREA_PUBLIC, 'm^2 Public Parcel Area,',round(AREA_PUBLIC/4046.8564224,2), 'acres (approximate)')

private_img_area = ee.Image.pixelArea().updateMask(ee.Image().paint(private_no_grass,1))
private_px_area = private_img_area.reduceRegion(**{
'reducer': ee.Reducer.sum(),
'geometry': REGION,
'scale': 5,
'crs': CRS,  
'maxPixels': 1e13
})
AREA_PRIVATE = ee.Number(ee.Dictionary(private_px_area).get('area')).getInfo()
print(AREA_PRIVATE, 'm^2 Private Parcel Area,', round(AREA_PRIVATE/4046.8564224,2), 'acres (approximate)')


TOTAL_AREA = AREA_PRIVATE + AREA_PUBLIC # ignores slightly overlapping areas b/w public and private, we treat the areas as mututally exclusive contributing toward the total
print(TOTAL_AREA, 'm^2 Total Area,', round(TOTAL_AREA/4046.8564224,2), 'acres (approximate)')

private_pct = round(AREA_PRIVATE/TOTAL_AREA,2)
public_pct = round(AREA_PUBLIC/TOTAL_AREA,2)
print(f"\nPrivate Parcels compose {private_pct} of Total Area\nPublic Parcels compose {public_pct} of Total Area")

549064867.7775726 m^2 Public Parcel Area, 135676.88 acres (approximate)
401810529.738924 m^2 Private Parcel Area, 99289.54 acres (approximate)
950875397.5164967 m^2 Total Area, 234966.43 acres (approximate)

Private Parcels compose 0.42 of Total Area
Public Parcels compose 0.58 of Total Area


## Required Functions, Run and Continue

In [5]:
seed=8

def treatment_math(TOTAL_AREA,DISTRO,SCENARIO_PCT_RANGE,SIZE_CLASSES):
    """Returns the amount of treatment units needed per size class and the radii of the kernel needed for each size class to make the randomized treatment landscape in EE
    # args:
    # TOTAL_AREA (int): total area of aoi to consider treatments in acres
    # DISTRO (str): distribution mode to use, one of: 'log', 'norm'
    # SCENARIO_PCT_RANGE (list): min and max range of possible pct treated area floats to be randomly chosen per scenario
    # SIZE_CLASSES (list): distinct acreage sizes to generate - list must be of length 4
    """
    dist_dct = {'log': [0.6,0.25,0.13,0.02], # probabilities of the given SIZE_CLASSES for each defined statistical distribution of treatments with acreages ranging 0-400
                'norm': [0.25,0.6,0.13,0.02]}
    
    # make list of length SCENARIOS with random floating point numbers in the range defined by SCENARIO_PCT_RANGE (list of different total pct treated scenarios)
    pct_trt = [i/100 for i in list(range(int(SCENARIO_PCT_RANGE[0]*100),int((SCENARIO_PCT_RANGE[1]+0.05)*100),5))] # intervals 0.05 - 0.6 intervals of 0.05
    #get approximate radius in meters needed for each size class to make correct-sized treatment units
    acres_to_sqm = [int(round(size*4046.8564224)) for size in SIZE_CLASSES] # convert to sq meters for each size class in acreage
    radii_circle = [int(round(math.sqrt(acreage)/math.pi)) for acreage in acres_to_sqm] # radius of circle: A = pi(r^2)  
    radii_square = [((math.sqrt(acreage)/2)) for acreage in acres_to_sqm] # rough square radius is A = (side/2) solve for side (side = sqrt(A) )
    
    # make lists of lists, one sub-list per scenario
    trt_areas=[]
    trt_props=[]
    units=[]
    for i in list(range(len(pct_trt))):
        
        # compute total area to be treated for each scenario
        trt_areas_i = (TOTAL_AREA/4046.8564224)*pct_trt[i] # convert total area to acreage then multiply by pct_treated random number (scenario i)
        trt_areas.append(trt_areas_i)
        
        pdf_probs = dist_dct[DISTRO] # grab the probabilities assigned to each tretment size class from the distribution dictionary
        
        i_prop = [round(p*trt_areas_i,2) for p in pdf_probs] # get acreage per size class as (pct of size class in distribution * total area to be treated)
        trt_props.insert(i,i_prop)
                
        units_i = [int(round(j)) for j in list(np.divide(i_prop,SIZE_CLASSES)) ]
        units.insert(i,units_i)
    
    return pct_trt,trt_areas,trt_props,units,radii_square

# EE functions
def distanceFilter(pts,distance):
    withinDistance = distance; 

    ## From the User Guide: https:#developers.google.com/earth-engine/joins_spatial
    ## add extra filter to eliminate self-matches
    distFilter = ee.Filter.And(ee.Filter.withinDistance(**{
      'distance': withinDistance,
      'leftField': '.geo',
      'rightField': '.geo', 
      'maxError': 1
    }), ee.Filter.notEquals(**{
      'leftField': 'system:index',
      'rightField': 'system:index',

    }));
    
    distSaveAll = ee.Join.saveAll(**{
                  'matchesKey': 'points',
                  'measureKey': 'distance'
    });
    # Apply the join.
    spatialJoined = distSaveAll.apply(pts, pts, distFilter);

    # Check the number of matches.
    # We're only interested if nmatches > 0.
    spatialJoined = spatialJoined.map(lambda f: f.set('nmatches', ee.List(f.get('points')).size()) );
    spatialJoined = spatialJoined.filterMetadata('nmatches', 'greater_than', 0);

    # The real matches are only half the total, because if p1.withinDistance(p2) then p2.withinDistance(p1)
    # Use some iterative logic to clean up
    def unpack(l): 
        return ee.List(l).map(lambda f: ee.Feature(f).id())

    def iterator_f(f,list):
        key = ee.Feature(f).id()
        list = ee.Algorithms.If(ee.List(list).contains(key), list, ee.List(list).cat(unpack(ee.List(f.get('points')))))
        return list
    
    ids = spatialJoined.iterate(iterator_f,ee.List([]))
    ##print("Removal candidates' IDs", ids);

    # Clean up 
    cleaned_pts = pts.filter(ee.Filter.inList('system:index', ids).Not());
    return cleaned_pts

def ee_treatments(aoi,pct_trt,units,radii,ownership):
    units = list(reversed(units)) # we generate treatment units in descending order of size
    radii = list(reversed(radii))
    
    # Overshoot multipliers - helps us sample the right ball park of extra points beyond the required # so that after the sample() fx drops nulls and we do the distance filtering routine, 
    # the pts are well spaced out over the AOI and are the correct required # for that size class
    default_overshoot=1.5
    sm_overshoot=2
    if pct_trt>0.55:
        sm_overshoot=4.85
    elif pct_trt>0.5:
        sm_overshoot=4.5
    elif pct_trt > 0.375:
        sm_overshoot=3.75
    elif pct_trt >0.275:
        sm_overshoot=3
    elif pct_trt>0.175:
        sm_overshoot=2.5
    med_overshoot=default_overshoot
    if pct_trt>0.175:
        med_overshoot=2
    
    mask_spacing = 1.1 # multiply by radius of given square (1.0 would be a square's full side length) these will also need to be dialed for a given amount of pct_trt on the landscpe so treatments look more or less evenly spaced out
    pt_spacing = 2.01 # multiply by radius of given square (2.0 would be a square's full side length)
    
    # Generate Biggest Treatments
    # sampling at a grid scale about half the desired square's radius (side/2) allows for sampled pts to automatically be more spaced out than if sampling at finer resolution, is more compute efficient
    ptsBiggest = ee.Image.constant(1).clip(aoi).sample(aoi,radii[0]/2,CRS,None,ee.Number(units[0]*default_overshoot).round(),seed,True,1,True).limit(units[0]) # apply overshoot to numPoints to get good ballpark of pts to try for before dropNulls and distanceFilter is applied
    # ptsBiggest_f = distanceFilter(ptsBiggest,radii[0]*pt_spacing).limit(units[0]) # so few of these that we don't need to use the distanceFilter route, just the limit() above
    ptsBiggestSize = ptsBiggest.size() # for QA
    areasBiggest = ee.Image().paint(ptsBiggest).Not().unmask(0).distance(ee.Kernel.euclidean(radii[0],'meters')).gte(0).clip(aoi).unmask(0).clip(REGION) # generate squares of desired size using the size class's radius

    # mask prohibiting Big pt samples within the overlap zone - Biggest areas plus buffer of Big treatments radius little bigger than radius so we don't have exact touching squares
    biggestMask = areasBiggest.distance(ee.Kernel.euclidean(radii[1]*mask_spacing,'meters')).gte(0).Not().unmask(1).clip(aoi) # reduce spacing requirement so that we can fit potentially as much as 90% of aoi area

    # Generate Big Treatments
    ptsBig = biggestMask.selfMask().sample(aoi,radii[1]/2,CRS,None,ee.Number(units[1]*default_overshoot).round(),seed,True,1,True) 
    ptsBig_f = distanceFilter(ptsBig,radii[1]*pt_spacing).limit(units[1])
    ptsBigSize= ptsBig_f.size()
    areasBig=ee.Image().paint(ptsBig).Not().unmask(0).distance(ee.Kernel.euclidean(radii[1],'meters')).gte(0).clip(aoi).unmask(0).clip(REGION) # the squares must be clipped to their specific aoi (public or private parcels) specifically before then clipping it to the export region

    # mask prohibiting Medium pt samples within overlap zone (Biggest and Big areas, plus a buffer of Medium treatments radius)
    biggestBigMask = areasBiggest.add(areasBig).gte(1).distance(ee.Kernel.euclidean(radii[2]*mask_spacing,'meters')).gte(0).Not().unmask(1).clip(aoi)

    # # # Generate Medium Treatments
    ptsMedium = biggestBigMask.selfMask().sample(aoi,radii[2]/2,CRS,None,ee.Number(units[2]*med_overshoot).round(),seed,True,1,True) 
    ptsMedium_f = distanceFilter(ptsMedium,radii[2]*pt_spacing).limit(units[2])
    ptsMediumSize= ptsMedium_f.size()
    areasMedium = ee.Image().paint(ptsMedium).Not().unmask(0).distance(ee.Kernel.euclidean(radii[2],'meters')).gte(0).clip(aoi).unmask(0).clip(REGION)

    # mask prohibiting Small pts samples within overlap zone (Biggest, Big, and Medium areas plus buffer of small treatments radius)
    biggestBigMediumMask = areasBiggest.add(areasBig).add(areasMedium).gte(1).distance(ee.Kernel.euclidean(radii[3]*mask_spacing,'meters')).gte(0).Not().unmask(1).clip(aoi) 

    # Generate Small Treatmetns
    ptsSmall = biggestBigMediumMask.selfMask().sample(aoi,radii[3]/2,CRS,None,ee.Number(units[3]*sm_overshoot).round(),seed,True,1,True) 
    ptsSmall_f = distanceFilter(ptsSmall,radii[3]*pt_spacing).limit(units[3]) # if sampling on a grid whose pixels are larger than the side of the desired square, the generated squares will never touch
    ptsSmallSize=ptsSmall_f.size()
    areasSmall = ee.Image().paint(ptsSmall_f).Not().unmask(0).distance(ee.Kernel.euclidean(radii[3],'meters')).gte(0).clip(aoi).unmask(0).clip(REGION) 

    blendTreatments = areasBiggest.add(areasBig).add(areasMedium).add(areasSmall).gte(1).multiply(DIST).selfMask().rename('DIST')
    
    treated_px_area = ee.Image.pixelArea().updateMask(blendTreatments.gte(1)).clip(REGION)
    
    total_trt_area = ee.Number(ee.Dictionary(
    treated_px_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': REGION,
    'scale': 5, 
    'crs': CRS,  
    'maxPixels': 1e13
    })).get('area'))

    pct_treated = ee.Number.parse(ee.String(ee.Number(total_trt_area.divide(TOTAL_AREA)).format('%.2f')))
    
    return (blendTreatments
            .set('reqBiggestPts',units[0],
            'actualBiggestPts',ptsBiggestSize,
            'reqBigPts',units[1],
            'actualBigPts',ptsBigSize,
            'reqMediumPts',units[2],
            'actualMediumPts',ptsMediumSize,
            'reqSmallPts',units[3],
            'actualSmallPts',ptsSmallSize,
            f'{ownership}_pct_trt_actual',pct_treated,
            'sm_overshoot',sm_overshoot,
            'med_overshoot',med_overshoot,
            'default_overshoot',default_overshoot)
           )

def export_img(img,imgcoll_p,aoi,scale):
    """Export image to imageCollection"""
    desc = f"scenario{ee.String(ee.Number(ee.Image(img).getNumber('scenario')).format()).getInfo()}"
    
    task = ee.batch.Export.image.toAsset(
        image=ee.Image(img).clip(aoi),
        description=desc,
        assetId=f'{imgcoll_p}/{desc}', 
        region=aoi, 
        scale=scale, 
        crs=CRS, 
        maxPixels=1e13)

    task.start()
    print(f"Export Started for {imgcoll_p}/{desc}")
    

## Generate Scenario Prescriptions

In [6]:
# Run treatment math to construct your lists (length of SCENARIO) of the needed parameters
pct_trt,trt_areas,trt_props,units,radii = treatment_math(TOTAL_AREA,DISTRO,SCENARIO_PCT_RANGE,SIZE_CLASSES)

# can look at your prescriptions by indexing any of the returned objects
print('Scenarios of Percent of Total Area to Treat:',pct_trt)
# print(trt_areas)
# print(trt_props)
# print(units)
# print(radii)

Scenarios of Percent of Total Area to Treat: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]


### Determine good threshold of Total Area percentage to treat that would get close to 90% of private parcels being treated

In [7]:
thresh = 0.35 # has to be one of the given pct_trt scenarios
pct_private = round((thresh*TOTAL_AREA)/AREA_PRIVATE,2)
print(f"{thresh} of Total Study Area constitutes {pct_private} of private parcel area\nBeyond this Percent Treated scenario, we will begin to generate remaining treatments in Public Parcels")

0.35 of Total Study Area constitutes 0.83 of private parcel area
Beyond this Percent Treated scenario, we will begin to generate remaining treatments in Public Parcels


## Construct random treatment landscapes as rasters - format: an ee.Image exported to an ee.ImageCollection

In [8]:
print(f'Static Parameters: TOTAL_AREA (ac) = {TOTAL_AREA/4046.8564224}; SCENARIO_PCT_RANGE = {SCENARIO_PCT_RANGE}; DISTRO = {DISTRO}; SIZE_CLASSES (ac) = {SIZE_CLASSES}, RADII PER SIZE CLASS (m) = {radii}')

# for each scenario, make the treated landscape raster 
today_string = datetime.utcnow().strftime("%Y-%m-%d").replace("-", "")

# make an ee.ImageCollection with specified path
runs_folder_path = f"projects/pyregence-ee/assets/aff-treatments/runs_scenarios_{len(list(range(len(pct_trt))))}_distro_{DISTRO}_privatePriority_{today_string}"
img_coll_p = f"{runs_folder_path}/treatment_scenarios"
print(f'Creating imgColl: {img_coll_p}')
os.popen(f"earthengine create folder {runs_folder_path}").read()
os.popen(f"earthengine create collection {img_coll_p}").read()

for i in list(range(len(pct_trt))):    # loop thru pct_trt elements, representing the scenarios
    print('\n')
    print('pct of Study Area to treat: ',pct_trt[i])
    print('total area (ac) to treat: ', trt_areas[i])    
    print('area per size class: ',trt_props[i])
    print('total units per size class: ', units[i])
   
    # if pct_trt[i] == 0:
    #     print('making empty trt img')
    #     empty_img = ee.Image.constant(0).clip(REGION).rename('DIST').set('scenario', i+1)
    #     export_img(empty_img, img_coll_p,REGION,EXPORT_SCALE)
    
    # start with private parcels, if pct_trt in the scenario is under the threshold (thresh determined above), only do treatments in private parcels
    # if pct_trt of the scenario is over given threshold, generate treatments in private parcels to the threshold, 
    # then use remainder of pct_trt over threshold to generate treatments in public parcels and merge the two together
    pct_private = round((pct_trt[i]*TOTAL_AREA)/AREA_PRIVATE,2)
    print(f"{pct_trt[i]} of Total Study Area constitutes {pct_private} of private parcel area")
    if pct_trt[i] >thresh:
        print('will divide treatments among private then public parcels')
        private_pct_trt = round(thresh/pct_trt[i],2) # proprotion of the total pct trt area to generate in private parcels
        print(private_pct_trt, 'of total treated area will be in private parcels')
        private_units= [round(i*private_pct_trt) for i in units[i]]
        print(private_units, 'private treatment units')
        
        public_pct_trt = round(1-private_pct_trt,2) # remaining proportion of total pct trt area to generate in public parcels
        print(public_pct_trt, 'of total treated area will be in public parcels')
        public_units = [round(i*public_pct_trt) for i in units[i]]
        print(public_units, 'public treatment units')
        
        trt_img_private = ee_treatments(private_p,private_pct_trt,private_units,radii,'private').set('scenario',i+1,'private_pct_trt_prescribed',private_pct_trt*pct_trt[i]) # want to diplay actual pct of the pct_trt that public/private will take up
        trt_img_public = ee_treatments(public_p,public_pct_trt,public_units,radii,'public').copyProperties(trt_img_private).set('public_pct_trt_prescribed',public_pct_trt*pct_trt[i]) 
        out_img = ee.Image(trt_img_public).unmask(0).add(ee.Image(trt_img_private).unmask(0)).gte(1).selfMask().multiply(DIST).copyProperties(trt_img_public)
        export_img(out_img,img_coll_p,REGION,EXPORT_SCALE) #export image to image collection
    else:
        print('will treat only in private parcels')
        trt_img_private = ee_treatments(private_p,pct_trt[i],units[i],radii,'private').set('scenario',i+1,'private_pct_trt_prescribed',pct_trt[i])
        export_img(trt_img_private,img_coll_p,REGION,EXPORT_SCALE) #export image to image collection
    
    #break


Static Parameters: TOTAL_AREA (ac) = 234966.42783105638; SCENARIO_PCT_RANGE = [0.0, 0.6]; DISTRO = log; SIZE_CLASSES (ac) = [10, 40, 100, 400], RADII PER SIZE CLASS (m) = [100.58454155584744, 201.16784037216286, 318.07467676632166, 636.1491570378759]
Creating imgColl: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_log_20220920/treatment_scenarios


pct of Study Area to treat:  0.0
total area (ac) to treat:  0.0
area per size class:  [0.0, 0.0, 0.0, 0.0]
total units per size class:  [0, 0, 0, 0]
0.0 of Total Study Area constitutes 0.0 of private parcel area
will treat only in private parcels
Export Started for projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_log_20220920/treatment_scenarios/scenario1


pct of Study Area to treat:  0.05
total area (ac) to treat:  11748.32139155282
area per size class:  [7048.99, 2937.08, 1527.28, 234.97]
total units per size class:  [705, 73, 15, 1]
0.05 of Total Study Area constitutes 0.12 of private parcel area

## Export Scenario Set to Google Drive

In [16]:
path = "projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios"
scn_imgColl = ee.ImageCollection(path)

toBands = scn_imgColl.sort('scenario').toBands()
#print(toBands.aggregate_array('scenario').getInfo())
fileNamePrefix = '_'.join(path.split('/')[-2:])
# print(desc)
desc = 'export_' + fileNamePrefix
folder = 'AFF_Fuelscapes'
task = ee.batch.Export.image.toDrive(image=toBands,description=desc,folder=folder,fileNamePrefix=fileNamePrefix,region=REGION,scale=EXPORT_SCALE,crs=CRS)
task.start()
print(f'Export started for {folder}/{fileNamePrefix}')


Export started for AFF_Fuelscapes/runs_scenarios_13_distro_norm_20220919_treatment_scenarios
